In [ ]:
from ratdata import process, ingest, data_manager as dm
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal

In [ ]:
tstart = 152 + 3
dur = 7
tend = tstart + dur

tstart_gui = 142.526 + 3
tend_gui = tstart_gui + dur
dur_gui = dur

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
dirname = Path("data/mce_recordings/")
filename = '2022-04-06T11-26-41 rat1 CT pro.mat'
gui_amplitude_filename = 'data/gui_recordings/2022-04-06T11-26-51-amplitude rat1.txt'

recording = ingest.read_mce_matlab_file(dirname / filename)
start = int(tstart / recording.dt)
end = int(tend / recording.dt)

data = np.mean(recording.electrode_data, axis=0)[start:end]
tt = np.linspace(10, 10 + dur, len(data))
biomarker, beta, total = process.dsp_biomarker(data)

gui_samples = int(200 * dur_gui)
gui_data = ingest.read_stim_amplitude_from_gui_recording(gui_amplitude_filename, 100)
tt_gui = np.linspace(10, 10 + dur_gui, gui_samples)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(18, 12))
axs[0].plot(tt, -data)
ax2 = axs[1]
shift = -50
biomarker2 = 0.7 * biomarker.copy()
# biomarker2 = biomarker2.clip(0, 100)
if shift == 0:
    ax2.plot(tt[::40], biomarker2[shift:], color='red')
elif shift < 0:
    ax2.plot(tt[(-shift * 40)::40], biomarker2[:shift], color='red')
else:
    ax2.plot(tt[:(-shift * 40):40], biomarker2[shift:], color='red')
# ax2.axhline(threshold, color='black')
# ax2.tick_params(axis='y', labelcolor='red')
gui_data_slice = gui_data[int(200 * tstart_gui): int(200 * tend_gui)]
axs[1].plot(tt_gui, gui_data_slice, color='black')
axs[0].set_title("LFP recording")
axs[0].set_ylabel("Amplitude [mV]")
axs[1].set_title("Relative beta and proportional controller amplitude")
axs[1].set_xlabel("Time [s]")
axs[1].set_ylabel("Stimulation amplitude [nA]")
axs[1].legend(["Relative beta", "Stim amplitude"])
plt.savefig('plots/biomarker_proportional.svg')
plt.savefig('plots/biomarker_proportional.png')

In [ ]:
tstart_onoff = 75
dur_onoff = 7.7
tend_onoff = tstart_onoff + dur_onoff

In [ ]:
filename = '2021-11-17T07-43-04 rat6 CT on-off.mat'

recording = ingest.read_mce_matlab_file(dirname / filename)
start = int(tstart_onoff / recording.dt)
end = int(tend_onoff / recording.dt)

data_onoff = np.mean(recording.electrode_data, axis=0)[start:end]
tt_onoff = np.linspace(10, 10 + dur_onoff, len(data_onoff))
biomarker_onoff, beta_onoff, total_onoff = process.dsp_biomarker(data_onoff)

In [ ]:
data_clipped = np.clip(data_onoff, 0, max(data_onoff))
spike_threshold = np.percentile(data_clipped, 99.5)
peaks = scipy.signal.find_peaks(data_clipped, spike_threshold)
peaks_ext = np.insert(peaks[0], -1, 0)
peaks_bursts = np.diff(tt_onoff[peaks_ext]) < 0.02

In [ ]:
starts = tt_onoff[peaks_ext[np.insert(np.roll(peaks_bursts, -1) & ~peaks_bursts, True, -1)]]
ends = tt_onoff[peaks_ext[np.insert(~np.roll(peaks_bursts, -1) & peaks_bursts, True, 0)]]

on_off_periods = np.zeros(len(tt_onoff))
for s, e in zip(starts, ends):
    on_off_periods[(tt_onoff > s) & (tt_onoff < e)] = 1

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(18, 12))
axs[0].plot(tt_onoff, data_onoff)
# axs[0].axhline(spike_threshold)
# axs[0].scatter(tt[peaks[0]], peaks[1]['peak_heights'], marker='v')
ax2 = axs[1]
shift = -50
biomarker2_onoff = biomarker_onoff.copy()
if shift == 0:
    ax2.plot(tt_onoff[::40], biomarker2_onoff, color='red')
elif shift < 0:
    ax2.plot(tt_onoff[(-shift * 40)::40], biomarker2_onoff[:shift], color='red')
else:
    ax2.plot(tt_onoff[:(-shift * 40):40], biomarker2_onoff[shift:], color='red')
# axs[1].axhline(50, color="black")
ax2.plot(tt_onoff, 100 * on_off_periods, color='black')
# ax2.tick_params(axis='y', labelcolor='red')
axs[0].set_title("LFP recording")
axs[0].set_ylabel("Amplitude [mV]")
axs[1].set_title("Stimulation amplitude and relative beta")
axs[1].set_xlabel("Time [s]")
axs[1].set_ylabel("Stimulation amplitude [nA]")
axs[1].legend(["Relative beta", "Stim amplitude"])
axs[0].set_xlim([10, 10 + dur_onoff])
axs[1].set_xlim([10, 10 + dur_onoff])
axs[1].set_ylim([-10, 150])

In [ ]:
fontsize_ax = 22
shift = -25

fig, axs = plt.subplots(2, 1, figsize=(18, 12))

lns = []

axs_right = [axs[0].twinx(), axs[1].twinx()]

l1 = axs[0].plot(tt_onoff, 100 * on_off_periods, color='black')
axs[0].set_ylim([-10, 180])
axs[0].set_xlim([10, 15.01])

if shift == 0:
    l2 = axs_right[0].plot(tt_onoff[::40], biomarker2_onoff / 10, color='red')
elif shift < 0:
    l2 = axs_right[0].plot(tt_onoff[(-shift * 40)::40], biomarker2_onoff[:shift] / 10, color='red')
else:
    l2 = axs_right[0].plot(tt_onoff[:(-shift * 40):40], biomarker2_onoff[shift:] / 10, color='red')

l3 = axs_right[0].axhline(5, color='purple', linestyle='--')

lns.append(l1 + l2 + [l3])
axs_right[0].set_ylim([-1, 22])

l1 = axs[1].plot(tt_gui, gui_data_slice, color='black')
axs[1].set_ylim([-10, 150])
axs[1].set_xlim([10, 15.01])

if shift == 0:
    l2 = axs_right[1].plot(tt[::40], biomarker2[shift:] / 0.7 / 10, color='red')
elif shift < 0:
    l2 = axs_right[1].plot(tt[(-shift * 40)::40], biomarker2[:shift] / 0.7 / 10, color='red')
else:
    l2 = axs_right[1].plot(tt[:(-shift * 40):40], biomarker2[shift:] / 0.7 / 10, color='red')

lns.append(l1 + l2)
axs_right[1].set_ylim([-1, 20])

for a in axs:
    a.tick_params(axis='both', which='major', labelsize=fontsize_ax)
    a.spines['top'].set_visible(False)
    a.spines['right'].set_visible(False)
    a.set_ylabel("Stimulation amplitude [$\mu$A]", fontsize=fontsize_ax)
    
for a in axs_right:
    a.tick_params(axis='both', which='major', labelsize=fontsize_ax, color="red", labelcolor="red")
    a.spines['top'].set_visible(False)
    a.spines['right'].set_visible(False)
    a.set_ylabel("Relative beta power [%]", fontsize=fontsize_ax, color="red")
    
axs[0].legend(lns[0], ["Stimulation amplitude", "Relative beta power", "Beta threshold"], fontsize=fontsize_ax, frameon=False, loc=2)
axs[1].legend(lns[1], ["Stimulation amplitude", "Relative beta power"], fontsize=fontsize_ax, frameon=False, loc=2)

axs[0].set_title('On-off control', fontsize=fontsize_ax + 10)
axs[0].text(9.9, 190, "A", fontsize=fontsize_ax + 10)
axs[1].set_title('Proportional control', fontsize=fontsize_ax + 10)
axs[1].text(9.9, 160, "B", fontsize=fontsize_ax + 10)
axs[1].set_xlabel('Time [s]', fontsize=fontsize_ax, labelpad=fontsize_ax)
plt.subplots_adjust(hspace=0.35)
plt.savefig(f'plots/beta_and_stim_amplitude_shift{shift}.png', bbox_inches="tight", facecolor="white")